Image_Based_Search_Functionality

In [ ]:
# Install DeepImageSearch if not already installed
!pip install DeepImageSearch --upgrade -q

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from DeepImageSearch import Load_Data, Search_Setup

# Define the folder containing the images
folder_path = '/content/Uploads'  # Change this to your folder path

# Function to resize images to 16:9 aspect ratio
def resize_image(image_path, target_width=640, target_height=360):
    img = cv2.imread(image_path)
    if img is not None:
        h, w = img.shape[:2]
        aspect_ratio = w / h

        if aspect_ratio > (target_width / target_height):
            new_width = target_width
            new_height = int(target_width / aspect_ratio)
        else:
            new_height = target_height
            new_width = int(target_height * aspect_ratio)

        resized_img = cv2.resize(img, (new_width, new_height))
        cv2.imwrite(image_path, resized_img)  # Overwrite the original image
    return image_path

# Load images from the specified folder and resize them
def load_images_from_folder(folder_path):
    image_list = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            full_path = os.path.join(folder_path, filename)
            resized_path = resize_image(full_path)
            image_list.append((resized_path, filename))
    return image_list

# Load images
image_list = load_images_from_folder(folder_path)

if not image_list:
    print("No images found in the specified folder.")
else:
    # Initialize the data loader
    dl = Load_Data()
    dl.from_folder([folder_path])

    # Initialize the search setup
    st = Search_Setup([img[0] for img in image_list], model_name="vgg19", pretrained=True)

    # Index images
    st.run_index()

    # Function to search for similar images
    def find_similar_images(image_path, num_images):
        similar_images = st.get_similar_images(image_path=image_path, number_of_images=num_images)
        return similar_images

    # Function to calculate Precision, Recall, and F1 Score
    def calculate_metrics(true_indices, retrieved_indices):
        true_positives = len(set(true_indices) & set(retrieved_indices))
        precision = true_positives / len(retrieved_indices) if retrieved_indices else 0
        recall = true_positives / len(true_indices) if true_indices else 0
        f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        return precision, recall, f1_score

    # Example usage
    query_image = "/content/Uploads/1727853617657-resized-1727853617650-youtube_4.jpeg"  # Change this to your specific query image path
    num_similar_images = 6

    # Assuming you have ground truth indices for the query image
    true_indices = [90, 91, 94, 92, 93]  # Replace with actual indices of relevant images
    similar_indices = find_similar_images(image_path=query_image, num_images=num_similar_images)

    # Print the paths and names of similar images
    print("Similar Images:")
    for index in similar_indices:
        img_path, img_name = image_list[index]
        print(f"Path: {img_path}, Name: {img_name}")

    # Calculate and print metrics
    precision, recall, f1 = calculate_metrics(true_indices, similar_indices)
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

    # Plot similar images
    st.plot_similar_images(image_path=query_image, number_of_images=num_similar_images)